In [9]:
import os
import csv
import cv2
from ultralytics import YOLO

MODEL_PATH = "BCSLens-models/yolo12n.pt"
IMAGE_FOLDER = "BCSLens-models/bcs_dataset"
OUTPUT_CSV = "prediction_results.csv"

model = YOLO(MODEL_PATH)

def predict_animal(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return []

    results = model.predict(img, imgsz=640)
    predictions = []

    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy()
        class_ids = r.boxes.cls.cpu().numpy()
        confidences = r.boxes.conf.cpu().numpy()

        for cls_id, conf, box in zip(class_ids, confidences, boxes):
            label = model.names[int(cls_id)]
            # กรองเฉพาะ dog หรือ cat
            if label.lower() in ["dog", "cat"]:
                predictions.append({
                    "label": label,
                    "confidence": float(conf),
                    "bbox": box.tolist()
                })
    return predictions

def predict_folder(image_folder, output_csv):
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["image_name", "label", "confidence", "bbox"])

        for img_name in image_files:
            img_path = os.path.join(image_folder, img_name)
            preds = predict_animal(img_path)

            if preds:
                for p in preds:
                    writer.writerow([img_name, p["label"], p["confidence"], p["bbox"]])
            else:
                writer.writerow([img_name, "No detection", "", ""])

            print(f"Processed {img_name}")

    print(f"Prediction completed. Results saved to {output_csv}")


if __name__ == "__main__":
    predict_folder(IMAGE_FOLDER, OUTPUT_CSV)



0: 480x640 1 dog, 73.6ms
Speed: 1.9ms preprocess, 73.6ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
Processed IMG_2565 - Pasavit TAPEN.jpeg

0: 480x640 1 person, 1 dog, 71.0ms
Speed: 1.5ms preprocess, 71.0ms inference, 11.9ms postprocess per image at shape (1, 3, 480, 640)
Processed IMG_20250726_083059 - Khomson Satchasataporn.jpg

0: 480x640 1 cat, 1 dog, 52.8ms
Speed: 2.0ms preprocess, 52.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Processed IMG_20250726_090434 - Khomson Satchasataporn.jpg

0: 480x640 3 persons, 1 dog, 51.7ms
Speed: 1.5ms preprocess, 51.7ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
Processed IMG_20250726_094326 - Khomson Satchasataporn.jpg

0: 640x480 2 persons, 1 dog, 2 bottles, 1 chair, 56.3ms
Speed: 1.7ms preprocess, 56.3ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)
Processed IMG_20250726_083110 - Khomson Satchasataporn.jpg

0: 480x640 1 person, 1 cat, 1 suitcase, 64.4ms
S